**Навигация по уроку**
1. [Как создать нейро-сотрудника?](https://colab.research.google.com/drive/1lEGaSx8gJyKHzrqnptLv7JA9w85fo8fT)
2. [Практическая реализация нейро-сотрудника](https://colab.research.google.com/drive/1g9YfsQFqrKCLgz8POS0Cnhtt-d2-JCLm)
3. Домашняя работа

В домашней работе вам предстоит придумать своего нейро-сотрудника.
Подумайте над личными данными сотрудника, кто его целевая группа, какие услуги он оказывает или какие задачи он решает. Составьте промпт для него.

**Для получения 3 баллов** за задание достаточно использовать простую базу-знаний (плохо структурированный гугл документ, любого объема).

**Для 4-х баллов**, необходимо структурировать гугл документ. В этом вам поможет ChatGPT, надо его об этом "попросить". Подумайте, как это лучше сделать? Оставьте комментарии по этому поводу в колабе с домашней работой.

Задание считается выполненным, если на входе языковой модели подаются фрагменты из векторной базы-данных в виде:
```
Заголовок 1 уровня (логическое описание, тема к которой относиться фрагмент)
Заголовок 2 уровня (отражает смысл фрагмента или группы, в которую входит фрагмент)
Фрагмент (из первоначального текста, либо оптимизированный chatGPT)
```
**Подсказка**. Попробуйте посмотреть на данные и составить к ним двух-уровневый план.

**Для 5 баллов** проведите оптимизацию нейро-сотрудника и опишите в свободной форме в колабе с домашней работой, что и как вы делали, а главное для чего.

# Тема работы

Некоторые размышления привели меня к мысли, что раз я изучаю нейросети, то было бы здорово иметь собственного помощника для систематизации моего пройденного курса и выполненных домашних заданий. Такой момент как раз представился в данном задании.

Стоит начать сразу с описания задуманного помощника, т.к. это является частью оптимизации проекта.

Предполагается, что ноутбук с домашней работой в colab создан как копия оригинального ноутбука с заданием, сохраняя в названии номер и текст "Домашняя работа".

Помощник будет спроектирован со следующими этапами, функциями и целями:

1) **Структурирование записей:**
- получение ссылок ноутбуков учетной записи в колабе:
    - авторизация на гугл драйв через API Google Colab Magic
    - запрос ноутбуков
- обработка каждой ссылки:
    - фильтрация блокнотов "untitled";
    - проверка поля "Владелец", чтобы определить, чей это ноутбук (колонка для развития проекта, т.к. в текущей версии видны только мои ноутбуки);
    - ~~проверка наличия текста "Домашняя работа" в названии  для выделения ноутбуков с домашними работами;~~
    - ~~проверка наличия заголовка "Итог" в теле ноутбука как индикатора завершенной работы. Данный заголовок в последующем стоит рассматривать как необходимый в домашних работах, т.к. этот проект может вырасти из статуса пет-проекта в полноценное приложение.~~
    - ~~если заголовок "Итог" отсутствует в блокноте домашней работы (см. пункт выше) - помечать блокнот как незаконченный.~~
    
    Обновлено:    
    - извлечение названия ноутбука;
    - извлечение навигации по уроку:
        - нахождение текста без ссылки в навигации к уроку и внесение его в колонку 'навигация';
    - извлечение всех заголовков ноутбука и внечение их в колонку 'Заголовки' в датафрейме;
    - извлечение текста после заголовка 'Итог', если такой имеется, если нет - отмечать его отстутствие, значение далее вносится в поле 'Итог' датафрейма;
   
- упорядочивание работ по номеру задания, которое берется из названия ноутбука;
- анализ кода в работах с помощью библиотеки ast:
    - извлечение классов;
    - извлечение функций;
    - передача в LLM класса/функции для получения краткого описания в виде
    ```
    Заголовок 1 уровня (Класс/Функция)
    Заголовок 2 уровня (логическая задача)
    Заголовок 3 уровня (краткое описание способа выполнения задачи)
    ```
     Предполагается использование LangChain для данного пункта.
     Для передачи в LLM сложных наборов классов/функций используется следующий метод:
        - 1. передача функций класса для получения описания
        - 2. получение описаний функций в классе от LLM
        - 3. передача в LLM класса с кратким описанием функций, полученных на предыдущем шаге
        - 4. получение описания класса
    - извлечение импортов;

- расчет хэша для ноутбуков;
- создание по итогу обработки найденных страниц из профиля документа-датафрейма с колонками:
    - 'ссылка': ссылка на ноутбук, метаданные в Chroma;
    - 'название': название ноутбука, документ в Chroma;
    - ~~задание;~~ ~~'навигация': название темы работы страницы из текстового блока навигации, документ в Chroma;~~
    - 'заголовки': текст заголовков любого уровня в уроке, документ в Chroma;
    - 'библиотеки': найденные импорты, метаданные в Chroma;
    - 'классы и функции': найденные классы и функции на странице, метаданные в Chroma;
    - 'описание': краткое описание от LLM классов и функций, документ в Chroma;
    - 'итог': итог выполненной работы, созданный из краткого описания текстового поля после заголовка "Итог" в работе, если такого поля нет - то так и добавляется значение "Нет", документ в Chroma;
    - 'хэш': хэш ноутбука, метаданные в Chroma.
- возможность сохранить и скачать документ локально в формате .csv и в гугл драйв по пути "/content/drive/MyDrive/assistant/database.csv".

2) **Создание векторной базы ChromaDB из структурированного документа:**
- создание базы из подготовленного датафрейма;
- индексирование документов в базе для последующего поиска;
- сохранение индексированной базы в гугл драйв пользователя с путем по умолчанию "/content/drive/MyDrive/assistant/chroma_db";
- загрузка индексов из гугл драйва для последующего использования.

3) **Создание ассистента для поиска данных в базе по запросам пользователя:**
- создать выбор моделей для работы ассистента;
- ассистент должен имееть функцию подсчета токенов через библиотеку tiktoken, необходимых для исполнения запроса к указанной модели;
- ассистент формирует запрос к выбранной LLM модели с параметрами:
    - промпт ассистента - поле с указанными параметрами по умолчанию с возможностью редактирования + найденные по схожести запроса пользователя документы в индексированной базе Chroma;
    - промпт запроса - поле с запросом пользователя;

4) **Создание интерфейса с использованием Gradio. Должен включать:**
- авторизацию в гугл драйв через всплывающее окно (временное решение, которое работает только при запуске в colab);
- поле с информацией об ассистенте:
    - название (тип) ассистента;
    - промпт ассистента;
- статус базы:
    - "не создана"/"создана"
    - поле для выбора пути для загрузки базы с указанным путем по умолчанию "/content/drive/MyDrive/assistant/chroma_db"
    - кнопка загрузки, которая подключает базу по указанному пути
    - статус меняется на "загружена", если использовалась успешная загрузка
    - статус меняется на "ошибка загрузки" при неудачной загрузке
- кнопка создания базы, если база не создана и не загружена
- область сохранения базы на локальное устройство:
    - поле с указанием пути до файла
    - кнопка сохранения: не активна, пока не существует датафрейма по пути гугл драйв (по умолчанию "/content/drive/MyDrive/assistant/database.csv")
    - кнопка сохранения: активна, когда файл по пути "/content/drive/MyDrive/assistant/database.csv" существует, либо указан путь к файлу.
- область запроса пользователя:
    - поле для ввода текста запроса с возможностью редактирования;
    - поле с ответом от ассистента:
        - ответ от LLM в текстовом виде
        - добавление метаданных из полей датафрейма, которые нашел ассистен в индексах: ссылка на ноутбук, название, задание, статус работы.
- область статистики:
    - количество использованных токенов в запросе
**Добавлено в процессе:**
- пргресс-бар от обработки до создания базы

**Возможные последующие доработки:**

Дообучение ассистента:
- возможность проверять драйв на появление новых ссылок ноутбуков и добавлять данные в датафрейм и векторную базу
- возможность изменять статус ноутбука
- возможность обновлять существующие записи при изменении ноутбука из базы (предположительно на основе изменения хэша ноутбуков)
    ...

## Эмбединги для OpenAI

*Небольшая справка по работе эмбединогов*

При использовании OpenAIEmbeddings в LangChain для кодирования текста в векторные представления (эмбеддинги) да, задействуется именно модель OpenAI, но не та, что генерирует текстовые ответы (например, GPT-4), а специализированные модели для эмбеддингов.

1. Какие модели используются?

OpenAI предоставляет отдельные embedding-модели, которые:

Оптимизированы для преобразования текста в векторы.

Не являются LLM (как GPT), но работают через тот же API.

Основные модели:

text-embedding-ada-002 (рекомендуемая, используется по умолчанию в LangChain).

Устаревшие: text-embedding-3-small, text-embedding-3-large.

2. Как это работает технически?

Отправка текста:

Ваш текст отправляется в API OpenAI embeddings, а не в chat/completions (как для GPT).
```
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = embeddings.embed_query("Пример текста")  # Возвращает список из 1536 чисел
```
Локальные вычисления:

Нет: Эмбеддинги вычисляются на серверах OpenAI, а не локально.

Словари/токенизаторы: OpenAI использует внутренние механизмы (например, BPE-токенизацию), но это скрыто от пользователя.

Отличие от LLM:

Эмбеддинг-модели не генерируют текст, а только преобразуют его в векторы.

Они дешевле и быстрее, чем GPT (text-embedding-ada-002 стоит $0.0001/1K токенов).

# Каркас с заглушками

In [ ]:
!pip install -q openai gradio tiktoken langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.4 MB/s eta 0:

In [ ]:
import ast
import hashlib
import pandas as pd
from typing import List, Dict, Optional
import chromadb
from langchain.schema import Document
from langchain.llms import OpenAI
import tiktoken
import gradio as gr

class NotebookProcessor:
    """Обрабатывает ноутбуки Google Colab: фильтрация, анализ кода, создание датафрейма."""

    def __init__(self):
        self.notebooks = []

    def authenticate(self) -> bool:
        """Авторизация в Google Drive через API.

        Returns:
            bool: True, если авторизация успешна.
        """
        # Реализация через Google Colab Magic (например, `from google.colab import drive; drive.mount('/content/drive')`)
        return True

    def fetch_notebooks(self) -> List[Dict]:
        """Загружает список ноутбуков из Google Drive.

        Returns:
            List[Dict]: Список ноутбуков с метаданными.
        """
        # Заглушка для примера
        return [{"name": "Домашняя работа 1", "url": "http://example.com/1", "owner": "user"}]

    def filter_notebooks(self, notebooks: List[Dict]) -> List[Dict]:
        """Фильтрует ноутбуки по критериям: название, владелец, наличие 'Домашняя работа'.

        Args:
            notebooks (List[Dict]): Список ноутбуков.

        Returns:
            List[Dict]: Отфильтрованный список.
        """
        return [nb for nb in notebooks if "Домашняя работа" in nb.get("name", "")]

    def parse_notebook_content(self, notebook_url: str) -> Dict:
        """Извлекает содержимое ноутбука: код, заголовки, секцию 'Итог'.

        Args:
            notebook_url (str): URL ноутбука.

        Returns:
            Dict: {'code': str, 'headers': List[str], 'has_итог': bool}.
        """
        # Заглушка: парсинг через API или файл
        return {"code": "def foo(): pass", "headers": ["Итог"], "has_итог": True}

    def analyze_code_with_ast(self, code: str) -> Dict:
        """Анализирует код с помощью AST.

        Args:
            code (str): Исходный код.

        Returns:
            Dict: {'functions': List[str], 'classes': List[str], 'imports': List[str]}.
        """
        tree = ast.parse(code)
        functions = [n.name for n in ast.walk(tree) if isinstance(n, ast.FunctionDef)]
        classes = [n.name for n in ast.walk(tree) if isinstance(n, ast.ClassDef)]
        imports = [n.names[0].name for n in ast.walk(tree) if isinstance(n, ast.Import)]
        return {"functions": functions, "classes": classes, "imports": imports}

    def generate_hash(self, content: str) -> str:
        """Генерирует хэш содержимого ноутбука.

        Args:
            content (str): Текст ноутбука.

        Returns:
            str: SHA-256 хэш.
        """
        return hashlib.sha256(content.encode()).hexdigest()

    def create_dataframe(self, notebooks_data: List[Dict]) -> pd.DataFrame:
        """Создает датафрейм с результатами анализа.

        Args:
            notebooks_data (List[Dict]): Данные ноутбуков.

        Returns:
            pd.DataFrame: Датафрейм с колонками по ТЗ.
        """
        df = pd.DataFrame(notebooks_data)
        df["status"] = df["has_итог"].apply(lambda x: "Завершена" if x else "Не завершена")
        return df

class LLMAnalyzer:
    """Анализирует код с помощью LLM: описание функций, подсчет токенов."""

    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.llm = OpenAI(model_name=model_name)
        self.tokenizer = tiktoken.get_encoding("cl100k_base")

    def generate_function_description(self, code: str) -> str:
        """Генерирует описание функции через LLM.

        Args:
            code (str): Исходный код функции.

        Returns:
            str: Описание в формате 'Заголовок 1 уровня\nЗаголовок 2 уровня'.
        """
        prompt = f"Опиши функцию:\n```python\n{code}\n```"
        response = self.llm(prompt)
        return response.strip()

    def count_tokens(self, text: str) -> int:
        """Подсчитывает количество токенов в тексте.

        Args:
            text (str): Входной текст.

        Returns:
            int: Число токенов.
        """
        return len(self.tokenizer.encode(text))

class ChromaDBManager:
    """Управляет векторной базой ChromaDB: создание, сохранение, поиск."""

    def __init__(self):
        self.client = chromadb.Client()
        self.collection = None

    def create_database(self, dataframe: pd.DataFrame):
        """Создает базу из датафрейма."""
        documents = dataframe["code"].tolist()
        metadatas = dataframe.to_dict("records")
        self.collection = self.client.create_collection("notebooks")
        self.collection.add(documents=documents, metadatas=metadatas)

    def search_documents(self, query: str, n_results: int = 3) -> List[Dict]:
        """Ищет документы по запросу."""
        return self.collection.query(query_texts=[query], n_results=n_results)

class Assistant:
    """Ядро ассистента: формирует запросы к LLM с контекстом из базы."""

    def __init__(self, db_manager: ChromaDBManager, llm_analyzer: LLMAnalyzer):
        self.db = db_manager
        self.llm = llm_analyzer

    def execute_query(self, user_query: str) -> str:
        """Выполняет запрос к LLM с данными из ChromaDB."""
        context = self.db.search_documents(user_query)
        prompt = f"Контекст:\n{context}\n\nЗапрос:\n{user_query}"
        return self.llm.llm(prompt)

class GradioInterface:
    """Интерфейс Gradio для взаимодействия с пользователем."""

    def __init__(self, assistant: Assistant):
        self.assistant = assistant

    def render_ui(self):
        """Создает интерфейс с полями ввода и вывода."""
        with gr.Blocks() as ui:
            gr.Markdown("## Ассистент для анализа ноутбуков Colab")
            with gr.Row():
                query = gr.Textbox(label="Ваш запрос")
                output = gr.Textbox(label="Ответ ассистента")
            query.submit(self.assistant.execute_query, inputs=query, outputs=output)
        return ui

## Порядок запуска

In [ ]:
# 1. Сначала импорты и настройки
imports_and_setup = """
!pip install -q openai gradio tiktoken langchain langchain-openai langchain-community chromadb ipywidgets
from google.colab import output
output.enable_custom_widget_manager()
"""

# 2. Затем объявления классов в правильном порядке:
classes_order = """
1. OperationResult (dataclass)
2. EnhancedCodeAnalyzer
3. Промпты (SYSTEM_PROMPT и др.)
4. LLMAnalyzer
5. ChromaDBManager
6. NotebookProcessor
7. Assistant
8. GradioInterface
"""

## Установка

In [ ]:
!pip install -q openai gradio tiktoken langchain langchain-openai langchain-community chromadb ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.4 MB/s eta 0:00:00


## Импорты

In [ ]:
import os
import re
import ast
import json
import traceback
import asyncio
import hashlib
import tiktoken
from collections import deque
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass

import pandas as pd
import chromadb
from chromadb.config import Settings
import gradio as gr
from google.colab import drive, userdata
import getpass
from googleapiclient.discovery import build
from IPython.display import display
import ipywidgets as widgets
from google.colab import output
output.enable_custom_widget_manager()

# LangChain импорты
from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# авторизация в драйве
from google.colab import drive
drive.mount('/content/drive')  # Здесь вы пройдёте авторизацию
from google.colab import auth
auth.authenticate_user()  # Токен для Google API

Mounted at /content/drive


## Класс OperationResult
для обработки результата выполнения функции и удобного вывода в интерфейс пользователя

In [ ]:
@dataclass
class OperationResult:
    success: bool
    message: str
    details: Optional[dict] = None

## Класс EnhancedCodeAnalyzer
вспомогательный класс для анализа кода и форматирования выводов

In [ ]:
class EnhancedCodeAnalyzer(ast.NodeVisitor):
    '''анализатор кода'''
    def __init__(self, source: str = None):
        self.source = source  # Сохраняем исходный код
        self.functions = []
        self.classes = []
        self.imports = []
        self.current_class = None

    def _get_source(self, node):
        """Безопасно получает исходный код для узла AST"""
        if self.source:
            try:
                return ast.get_source_segment(self.source, node) or f"def {node.name}(...)"
            except:
                return f"def {node.name}(...)"
        return f"def {node.name}(...)"

    def visit_ClassDef(self, node):
        self.classes.append(node.name)
        self.current_class = node.name
        self.generic_visit(node)
        self.current_class = None

    def visit_FunctionDef(self, node):
        try:
            func_info = {
                'name': node.name,
                'args': [arg.arg for arg in node.args.args],
                'lineno': node.lineno,
                'code': self._get_source(node)
            }
            if self.current_class:
                func_info['class_name'] = self.current_class
            self.functions.append(func_info)
        except Exception as e:
            func_info['code'] = f"def {node.name}(...)"  # Fallback

    def visit_Import(self, node):
        for alias in node.names:
            self.imports.append(alias.name)

    def visit_ImportFrom(self, node):
        module = node.module
        for alias in node.names:
            self.imports.append(f"{module}.{alias.name}")

## Класс LLMAnalyzer
Отправляет в LLM классы и функции для получения краткого описания в формате
```
    Заголовок 1 уровня (Функция)
    Заголовок 2 уровня (логическая задача функции)
    Заголовок 3 уровня (краткое описание способа достижения задачи функции)
```

In [ ]:
SYSTEM_PROMPT = """Ты - ассистент для анализа ноутбуков Colab. Отвечай точно, используя контекст. Будь лаконичным."""

FUNCTION_DESC_PROMPT = """Сгенерируй описание функции:\n{code}\n---\nФормат:
# Название функции
## задача функции
### краткое описание способа выполнения задачи (1-2 коротких предложения)"""

CLASS_DESC_PROMPT = """Сгенерируй краткое описание класса согласно шаблону:

# Название класса (основная цель)
## Общая архитектура (только перечисление названий компонентов)
### Взаимодействие компонентов

Класс:
{code}

Описания методов:
{methods_descriptions}"""

In [ ]:
def get_api_key():
    """Получает или сохраняет API ключ в секретах Colab"""
    try:
        # Пробуем получить существующие секреты
        api_key = userdata.get('LLM_API_KEY')
        api_base = userdata.get('LLM_API_BASE')
        return api_key, api_base
    except userdata.SecretNotFoundError:
        # Если секретов нет - запрашиваем и сохраняем
        print("Секреты не найдены. Введите данные API:")
        api_key = getpass.getpass("API ключ: ").strip()
        api_base = input("API адрес (по умолчанию https://api.openai.com/v1): ").strip() or 'https://api.openai.com/v1'
        return api_key, api_base

# Инициализация с запросом ключа
API_KEY, API_BASE = get_api_key()

In [ ]:
class LLMAnalyzer:
    """Анализирует код с помощью LLM: описание функций, подсчет токенов."""

    def __init__(self, openai_api_key: str, openai_api_base: str = None, model_name: str = "gpt-4.1-mini"):
        self.llm = ChatOpenAI(
            openai_api_key=openai_api_key,
            openai_api_base=openai_api_base if openai_api_base else None,
            model_name=model_name,
            temperature=0.3
            )
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        self.embeddings = OpenAIEmbeddings(
            openai_api_key=openai_api_key,
            openai_api_base=openai_api_base if openai_api_base else None,
            model="text-embedding-3-small"  # Рекомендуемая модель
        )
        self.system_prompt = SYSTEM_PROMPT
        self._init_prompts()

    def _init_prompts(self):
        """Инициализация промптов для LangChain."""
        # Промпт для пользовательских запросов (RAG)
        self.qa_prompt = PromptTemplate(
            input_variables=["system_prompt", "context", "query"],
            template="""
            {system_prompt}
            Контекст: {context}
            Запрос: {query}
            Ответь кратко, используя только релевантные данные.
            """
        )

    def update_system_prompt(self, new_prompt: str) -> None:
        """Обновляет системный промпт"""
        self.system_prompt = new_prompt

    def generate_response(self, user_query: str, context: str) -> tuple[str, int]:
        """Генерирует ответ через LangChain (пользовательские запросы)."""
        try:
            chain = LLMChain(
                llm=self.llm,
                prompt=self.qa_prompt
            )
            response = chain.run({
                "system_prompt": self.system_prompt,
                "context": context,
                "query": user_query
                })
            tokens = self.count_tokens(response)
            return response.strip(), tokens

        except Exception as e:
            logger.error(f"Ошибка генерации ответа: {str(e)}")
            return f"Ошибка: {str(e)}", 0

    def count_tokens(self, text: str) -> int:
        """Подсчет токенов в тексте"""
        return len(self.tokenizer.encode(text))

    async def _describe_code_element(self, element: Dict) -> OperationResult:
        '''генерирует описание функций'''
        try:
            # Если element - строка с кодом (прямой вызов)
            if isinstance(element, str):
                return await self._describe_function(element)

            # Если element - словарь из AST-анализа
            code = element.get('code', '')

            if not code:
                return OperationResult(
                    success=False,
                    message='Пустой код для описания',
                    details={'element': element}
                )

            if 'class_name' in element: # описание метода для класса
                return await self._describe_function(element["code"])

            elif 'def' in element['code'][:4]: #самостоятельная функция
                return await self._describe_function(element['code'])

            else:
                return await self._describe_class(element)

        except Exception as e:
            logger.error(f"Ошибка генерации описания: {str(e)}\nElement: {element}")
            return OperationResult(
                success=False,
                message='Ошибка генерации описания',
                details={
                    'error': str(e),
                    'element_type': type(element).__name__,
                    'element_content': str(element)[:200]  # Логируем часть содержимого
                }
            )

    async def _describe_function(self, code: str) -> OperationResult:
        '''описание функции'''
        prompt = FUNCTION_DESC_PROMPT

        try:
            response = await self.llm.agenerate([prompt.format(code=code)])
            return OperationResult(
                success=True,
                message="Описание функции сгенерировано",
                details={
                    "description": response.generations[0][0].text,
                    "tokens_used": response.llm_output['token_usage']['total_tokens']
                }
            )
        except Exception as e:
            return OperationResult(
                success=False,
                message="Ошибка описания функции",
                details={"error": str(e)}
            )

    async def _describe_class(self, class_info: Dict) -> OperationResult:
        """Генерирует описание для класса"""
        # Сначала получаем описания методов
        methods_descriptions = []
        for method in class_info['methods']:
            result = await self._describe_function(method['code'])
            if result.success:
                methods_descriptions.append(
                    f"{method['code']}\n{result.details['description']}"
                )

        # Формируем промпт для класса
        class_prompt = CLASS_DESC_PROMPT.format(
            code=class_info['code'],
            methods_descriptions="\n\n".join(methods_descriptions)
        )

        try:
            response = await self.llm.agenerate([class_prompt])
            return OperationResult(
                success=True,
                message="Описание класса сгенерировано",
                details={
                    "description": response.generations[0][0].text.strip(),
                    "tokens_used": response.llm_output['token_usage']['total_tokens']
                }
            )
        except Exception as e:
            return OperationResult(
                success=False,
                message="Ошибка описания класса",
                details={"error": str(e)}
            )

## Класс ChromaDBManager
Работает с базой данных: настраивает Chroma, создает, загружает, сохраняет базу, ищет документы, форматирует вывод.

In [ ]:
class ChromaDBManager:
    """Управление векторной базой ChromaDB с поддержкой нового интерфейса эмбеддингов"""

    def __init__(self, persist_dir: Optional[str] = None, embeddings: OpenAIEmbeddings = None):
        self.embeddings = embeddings
        self._persist_dir = persist_dir
        self._client = None
        self._collection = None

    @property
    def client(self):
        '''Инициализация клиента'''
        if self._client is None:
            if self._persist_dir and "/content/drive" in self._persist_dir:
                self._ensure_drive_mounted()

            self._client = chromadb.PersistentClient(path=self._persist_dir) if self._persist_dir \
                          else chromadb.EphemeralClient()
        return self._client

    @property
    def collection(self):
        '''Инициализация коллекции с новым интерфейсом эмбеддингов'''
        if self._collection is None:
            class ChromaCompatibleEmbeddingFunction:
                def __init__(self, embeddings):
                    self._embeddings = embeddings

                # Новый интерфейс с параметром 'input' вместо 'texts'
                def __call__(self, input: List[str]) -> List[List[float]]:
                    if self._embeddings:
                        embeddings = self._embeddings.embed_documents(input)
                        return [e.tolist() if hasattr(e, 'tolist') else e for e in embeddings]
                    return [[0.0]*384 for _ in input]  # Заглушка

                def name(self):
                    return "custom_embedding"

            embedding_fn = ChromaCompatibleEmbeddingFunction(self.embeddings)

            try:
                self._collection = self.client.get_or_create_collection(
                    name="notebooks",
                    embedding_function=embedding_fn
                )
            except Exception as e:
                logger.error(f"Ошибка при создании коллекции: {str(e)}")
                raise
        return self._collection

    @staticmethod
    def _ensure_drive_mounted():
        '''Проверка подключения Google Drive'''
        if not os.path.exists('/content/drive/MyDrive'):
            drive.mount('/content/drive')
            assert os.path.exists('/content/drive/MyDrive'), "Google Drive не подключен!"

    def create_database(self, dataframe: pd.DataFrame, progress_callback: Optional[callable] = None) -> OperationResult:
        """Создание базы данных с обработкой ошибок"""
        try:
            # Подготовка данных
            documents = []
            metadatas = []
            ids = [str(i) for i in range(len(dataframe))]

            for i, (_, row) in enumerate(dataframe.iterrows()):
                doc = f"Название: {row['название']}\nЗаголовки: {row['заголовки']}\nОписание: {row['описание']}\nИтог: {row['итог']}"
                documents.append(doc)
                metadatas.append(row.to_dict())

                if progress_callback:
                    progress = 70 + int((i + 1) / len(dataframe) * 30)
                    progress_callback(progress)

            # Добавление данных
            self.collection.add(
                documents=documents,
                metadatas=metadatas,
                ids=ids
            )

            return OperationResult(
                success=True,
                message=f"База успешно создана. Документов: {len(documents)}",
                details={"path": self._persist_dir}
            )

        except Exception as e:
            logger.error(f"Критическая ошибка: {str(e)}", exc_info=True)
            return OperationResult(
                success=False,
                message="Не удалось создать базу данных",
                details={
                    "error_type": type(e).__name__,
                    "error_msg": str(e),
                    "solution": (
                        "1. Проверьте версию ChromaDB (должна быть 0.4.16+)\n"
                        "2. Убедитесь в правильности API ключей\n"
                        "3. Попробуйте другой путь для базы"
                    )
                }
            )

    def as_retriever(self, k: int = 3):
        '''совместимость с RetrievealQA'''
        if not self.collection:
            raise ValueError('База не загружена')
        return self.collection.as_retriever(search_kwargs={'k': k})

    def search_documents(self, query: str, n_results: int = 3,
                         filter_conditions: Optional[Dict] = None) -> OperationResult:
        """Расширенный поиск с фильтрацией."""
        try:
            results = self.collection.querry(
                query_texts=[query],
                n_results=n_results,
                where=filter_conditions
            )
            formatted = self._format_results(results)
            return OperationResult(
                success=True,
                message=f"Найдено {len(formatted)} результатов",
                details={
                    "results": formatted,
                    "query": query,
                    "filters": filter_conditions
                }
            )
        except Exception as e:
            return OperationResult(
                success=False,
                message="Ошибка при выполнении поиска",
                details={
                    "error_type": type(e).__name__,
                    "error_msg": str(e),
                    "query": query,
                }
            )

    def _format_results(self, raw_results: Dict) -> List[Dict]:
        '''Форматирует результаты поиска для Gradio'''
        formatted = []
        for i in range(len(raw_results['ids'][0])):
            formatted.append({
                "id": raw_results['documents'][0][i],
                "document": raw_results['documents'][0][i],
                "metadata": raw_results['documents'][0][i],
                "distance": raw_results['documents'][0][i]
            })
        return formatted

    def save_database(self, path: str) -> OperationResult:
        '''Сохраняет базу на диск с обновлением пути'''
        try:
            self._persist_dir = path
            self.clinet = chromadb.Client(self._make_settings(path))
            self.client.persist()
            return OperationResult(
                success=True,
                message=f"База сохранена в {path}",
                details={"path": path}
            )
        except Exception as e:
            return OperationResult(
                success=False,
                message='Ошбика сохранения базы',
                details={
                    "error_type": type(e).__name__,
                    "error_msg": str(e),
                    }
            )

    def load_database(self, path: str, collection_name: str = 'notebooks') -> OperationResult:
        '''загружает базу с диска'''
        try:
            self._persist_dir = path
            self.client = chromadb.Client(self._make_settings(path))
            self.collection = self.client.get_collection(collection_name)
            return OperationResult(
                success=True,
                message='База загружена',
                )
        except Exception as e:
            return OperationResult(
                success=False,
                message='Ошбика загрузки базы',
                details={
                    "error_type": type(e).__name__,
                    "error_msg": str(e),
                    }
            )

## Класс NotebookProcessor
Обрабатывает ноутбуки Colab по ТЗ

In [ ]:
class NotebookProcessor:
    """Обрабатывает ноутбуки Google Colab: фильтрация, извлечение кода, создание датафрейма."""

    def __init__(self, llm_analyzer: LLMAnalyzer):
        self.notebooks = []
        self.llm_analyzer = llm_analyzer
        self.drive_service = build('drive', 'v3')

    def fetch_notebooks(self) -> OperationResult:
        """Загружает список ноутбуков из Google Drive"""
        try:
            results = self.drive_service.files().list(
                q="mimeType='application/vnd.google.colaboratory' and not name contains 'untitled'",
                fields='files(id, name, webViewLink, owners)',
                pageSize=1000
            ).execute()

            notebooks = [{
                'id': item['id'],
                'name': item['name'],
                'url': item.get('webViewLink', ''),
                'owner': item.get('owners', [{}])[0].get('emailAddress') if item.get('owners') else None
            } for item in results.get('files', [])]

            return OperationResult(
                success=True,
                message=f'Найдено {len(notebooks)} ноутбуков',
                details={'notebooks': notebooks}
            )

        except Exception as e:
            logger.error(f"Ошибка получения ноутбуков: {str(e)}")
            return OperationResult(
                success=False,
                message='Ошибка получения ноутбуков',
                details={
                    'error': str(e),
                    'solution': 'Проверьте: 1) drive.mount() 2) auth.authenticate_user()'
                }
            )

    def _extract_number(self, name: str) -> float:
        '''извлекает номер урока'''
        try:
            match = re.search(r'(\d{1,2}\.\d{1})', name)
            if match:
                return float(match.group(1))
            return 0.0
        except ValueError:
            logger.error(f"Ошибка извлечения номера из '{name}': {str(e)}")
            return 0.0

    def sorted_notebooks(self, notebooks: List[Dict]) -> OperationResult:
        """Сортирует ноутбуки по номеру урока"""
        sorted_nbs = []
        for nb in notebooks:
            nb['number'] = self._extract_number(nb['name'])
            sorted_nbs.append(nb)

        sorted_nbs.sort(key=lambda x:x['number'])

        return OperationResult(
            success=True,
            message=f"Ноутбуки отсортированыы",
            details={"notebooks": sorted_nbs}
        )

    def extract_headers(self, notebook_json: dict) -> str:
        '''извлекает заголовки в ноутбуке
            Учитывает:
            - Заголовки с пробелами и без (#Text и # Text)
            - Только в начале строки (не в середине текста)
            - Игнорирует комментарии в коде
        '''
        try:
            # JSON-формат
            headers = []
            for cell in notebook_json.get('cells', []):
                if cell['cell_type'] == 'markdown':
                    for line in cell['source']:
                        line = line.strip()
                        if line.startswith('#') and len(line) > 1:
                        # Удаляем все # в начале и лишние пробелы
                            header = re.sub(r'^#+\s*', '', line)
                            if header:
                                headers.append(header)
            return '\n'.join(headers) if headers else 'нет заголовков'

        except Exception as e:
            logger.error(f"Ошибка извлечения заголовков: {str(e)}")
            return "Ошибка обработки заголовков"

    def extract_summary(self, notebook_json: dict) -> str:
        '''извлекает поле Итог'''
        try:
            for cell in notebook_json.get('cells', []):
                if cell['cell_type'] == 'markdown':
                    cell_text = ''.join(cell['source'])
                    if re.search(r'^#+\s*Итог\s*$', cell_text, re.IGNORECASE | re.MULTILINE):
                        # Берем все после заголовка до следующего заголовка
                        parts = re.split(r'^#+\s*Итог\s*$', cell_text, flags=re.IGNORECASE | re.MULTILINE)
                        if len(parts) > 1:
                            summary = parts[1].strip()
                            # Удаляем последующие заголовки
                            summary = re.split(r'^#+', summary, flags=re.MULTILINE)[0].strip()
                            return summary if summary else "Пустой итог"
            return "Нет итога"
        except Exception as e:
            logger.error(f"Ошибка извлечения итога: {str(e)}")
            return f"Ошибка извлечения итогов: {str(e)}"

    def _format_code_elements(self, analysis: Dict) -> str:
        '''форматирует отображение в виде
            класс:
            # функция

            функция
        '''
        # классы с методами
        elements=[]
        for class_name in analysis['classes']:
            class_methods=[
                f"# def {method['name']}({', '.join(method['args'])})]"
                for method in analysis['functions']
                if method.get('class_name') == class_name
            ]
            if class_methods:
                elements.append(f"class {class_name}:")
                elements.extend(class_methods)
        # самостоятельные функции
        standalone_funcs = [
            f"def {func['name']}({', '.join(func['args'])})"
            for func in analysis['functions']
            if not func.get("class_name")
        ]
        elements.extend(standalone_funcs)

        return '\n'.join(elements) if elements else 'нет классов/функций'

    def analyze_code_with_ast(self, code: str) -> Dict:
        """Анализирует код с помощью AST с обработкой ошибок синтаксиса"""
        try:
            tree = ast.parse(code)
        except SyntaxError as e:
            # Логируем ошибку, но продолжаем обработку
            logger.warning(f"Синтаксическая ошибка в коде: {str(e)}")
            return {
                "imports": [],
                "classes": [],
                "functions": [],
                "syntax_error": str(e)
            }
        except IndentationError as e:
            logger.warning(f"Ошибка отступов в коде: {str(e)}")
            return {
                "imports": [],
                "classes": [],
                "functions": [],
                "indentation_error": str(e)
            }
        except Exception as e:
            logger.warning(f"Другая ошибка при анализе кода: {str(e)}")
            return {
                "imports": [],
                "classes": [],
                "functions": []
            }

        analyzer = EnhancedCodeAnalyzer(source=code)
        analyzer.visit(tree)
        return {
            "imports": analyzer.imports,
            "classes": analyzer.classes,
            "functions": analyzer.functions
        }

    def generate_hash(self, content: str) -> str:
        """Генерирует хэш содержимого ноутбука"""
        return hashlib.sha256(content.encode()).hexdigest()

    async def process_notebooks(self, progress_callback=None) -> OperationResult:
        '''основной метод обработки всех ноутбуков'''
        try:
            # Этап 1: Загрузка и сортировка (0-20%)
            if progress_callback is not None:
                if asyncio.iscoroutinefunction(progress_callback):
                    await progress_callback(5)
                else:
                    progress_callback(5)

            # получаем и сортируем
            fetch_result = self.fetch_notebooks()
            if not fetch_result.success:
                return fetch_result

            if progress_callback is not None:
                if asyncio.iscoroutinefunction(progress_callback):
                    await progress_callback(10)
                else:
                    progress_callback(10)

            sort_results = self.sorted_notebooks(fetch_result.details['notebooks'])
            total = len(sort_results.details['notebooks'])

            if progress_callback is not None:
                if asyncio.iscoroutinefunction(progress_callback):
                    await progress_callback(20)
                else:
                    progress_callback(20)

            notebooks_data = []
            for i, nb in enumerate(sort_results.details['notebooks']):
                # загружаем содержимое ноутбука
                try:
                    request = self.drive_service.files().get_media(fileId=nb['id'])
                    file_content = request.execute()

                    # Декодируем содержимое (для текстовых файлов)
                    notebook_content = file_content.decode('utf-8') if isinstance(file_content, bytes) else file_content
                    notebook_json = json.loads(notebook_content)

                    # анализ кода из всех ячеек с фильтрацией
                    all_code = '\n'.join(
                        '\n'.join(
                            line for line in cell['source']
                            if not (
                                line.strip().startswith('%') or      # Магические команды (%matplotlib)
                                line.strip().startswith('!') or      # Shell-команды (!pip install)
                                line.strip().startswith('# @') or    # Специальные аннотации (# @title)
                                line.strip().startswith('?') or         # Справка (?str)
                                line.strip().startswith('"""') or    # Многострочные строки
                                line.strip().startswith("'''")       # Многострочные строки
                            )
                        )
                        for cell in notebook_json.get('cells', [])
                        if cell['cell_type'] == 'code'
                        and not any(  # Пропускаем ячейки с определенными метаданными
                            meta.get('colab_type') == 'example' or
                            meta.get('executionInfo', {}).get('status') == 'aborted'
                            for meta in [cell.get('metadata', {})]
                        )
                    )
                    code_analysis = self.analyze_code_with_ast(all_code)

                    # генерация описания
                    descriptions = []
                    for item in code_analysis['functions'] + code_analysis['classes']:
                        result = await self.llm_analyzer._describe_code_element(item)
                        if result.success:
                            descriptions.append(result.details['description'])

                    notebooks_data.append({
                        'ссылка': nb['url'],
                        'название': nb['name'],
                        'заголовки': self.extract_headers(notebook_json),
                        'библиотеки': ', '.join(code_analysis['imports']),
                        'классы и функции': self._format_code_elements(code_analysis),
                        'описание': '\n\n'.join(descriptions),
                        'итог': self.extract_summary(notebook_json),
                        'хэш': self.generate_hash(str(notebook_json))
                    })

                    if progress_callback is not None:
                        progress = 20 + int((i + 1) / total * 50)  # 20-70%
                        if asyncio.iscoroutinefunction(progress_callback):
                            await progress_callback(progress)
                        else:
                            progress_callback(progress)

                except Exception as e:
                    logger.error(f"Ошибка обработки ноутбука {nb.get('name', '')}: {str(e)}")
                    continue

            df = pd.DataFrame(notebooks_data)
            return OperationResult(
                success=True,
                message=f'Обработано {len(df)} ноутбуков',
                details={'dataframe': df}
            )
        except Exception as e:
            logger.critical(f"Критическая ошибка: {str(e)}", exc_info=True)
            return OperationResult(
                success=False,
                message='Ошибка обработки ноутбуков',
                details={
                    'error_type': type(e).__name__,
                    'error_msg': str(e)
                }
            )

## Класс Assistant
Отправляет запросы пользователя с найденными сходствами из базы в LLM и принимает ответы

In [ ]:
class Assistant:
    """Ядро ассистента с поддержкой системного промпта и поиска в базе"""

    def __init__(self, db_manager: ChromaDBManager, llm_analyzer: LLMAnalyzer):
        self.db = db_manager
        self.llm = llm_analyzer
        self.qa_chain = None
        self.current_system_prompt = llm_analyzer.system_prompt

    def update_system_prompt(self, new_prompt: str) -> None:
        """Обновляет системный промпт"""
        self.current_system_prompt = new_prompt
        self.llm.update_system_prompt(new_prompt)

    def execute_query(self, query: str) -> tuple[str, int]:
        """Выполняет запрос с поиском в базе и учетом системного промпта"""
        try:
            if not self.qa_chain:
                self.qa_chain = RetrievalQA.from_chain_type(
                    llm=self.llm.llm,
                    chain_type="stuff",
                    retriever=self.db.as_retriever(),
                    chain_type_kwargs={
                        "prompt": self.llm.qa_prompt
                    }
                )
            response = self.qa_chain.run(query)
            return response, self.llm.count_tokens(response)
        except Exception as e:
            logger.error(f"Ошибка запроса ассистена: {str(e)}")
            return f"Ошибка: {str(e)}", 0

## Интерфейс Gradio
Взаимодействие пользователя с Ассистентом, вывод всех результатов операций, включая внутренние ошибки.

In [ ]:
class GradioInterface:
    '''Интерфейс Gradio с интеграцией ассистента'''

    def __init__(self, processor: NotebookProcessor, db_manager: ChromaDBManager, assistant: Assistant, ):
        self.processor = processor
        self.assistant = assistant
        self.db_manager = db_manager
        self.db_status = "не создана"
        self.default_db_path = "/content/drive/MyDrive/assistant/chroma_db"
        self.default_csv_path = "/content/drive/MyDrive/assistant/database.csv"
        self._progress = 0
        self._status = ""
        self._progress_callbacks = []
        self._loop = asyncio.get_event_loop()

    async def _update_progress(self, progress: int, message: str):
        """Обновляет прогресс и уведомляет все callback-функции"""
        self._progress = progress
        self._status = message
        for callback in self._progress_callbacks:
            try:
                if asyncio.iscoroutinefunction(callback):
                    await callback(progress, message)
                else:
                    callback(progress, message)
            except Exception as e:
                logger.error(f"Ошибка в callback прогресса: {str(e)}")

    def add_progress_callback(self, callback):
        """Добавляет callback для обновления прогресса"""
        self._progress_callbacks.append(callback)

    async def _db_operation(self, operation: str, db_path: str, progress_callback: Optional[callable]=None) -> str:
        '''обработчик операций с базой данных'''
        try:
            logger.info(f"Начало операции: {operation} с базой по пути: {db_path}")
            # Инициализация прогресса
            if progress_callback:
                progress_callback(0)

            if operation =='create':
                logger.info("Этап 1: обработка ноутбуков")
                if progress_callback:
                    progress_callback(10)



                # Этап 1: Обработка ноутбуков
                process_result = await self.processor.process_notebooks()

                if not process_result.success:
                    error_msg = "Ошибка обработки:\n{}".format(process_result.message)
                    if process_result.details:
                        error_msg += "\nДетали:\n{}".format(
                            "\n".join(f"{k}: {v}" for k, v in process_result.details.items())
                        )
                    logger.error(error_msg)
                    logger.error(process_result.details)
                    return error_msg

                # Этап 2: создание базы
                if progress_callback:
                    progress_callback(70)

                result = self.db_manager.create_database(
                    dataframe=process_result.details['dataframe']
                    )

            if progress_callback:
                    progress_callback(100)

        except Exception as e:
            logger.error("Ошибка в _db_operation", exc_info=True)
            error_msg = "⚠️ Критическая ошибка:\n• Тип: {}\n• Сообщение: {}".format(
                type(e).__name__, str(e))
            return error_msg

    async def _create_db(self):
        """Асинхронный процесс создания базы данных"""
        try:
            # Проверяем авторизацию
            if not os.path.exists('/content/drive/MyDrive'):
                yield [gr.Slider(visible=False), "❌ Ошибка: Google Drive не подключен"]
                return

            # Инициализация прогресса
            await self._update_progress(0, "🔄 Подготовка...")
            yield [gr.Slider(visible=True, value=0), "🔄 Подготовка..."]

            # Создаем отдельную задачу для обработки ноутбуков
            async def process_notebooks_task():
                def progress_callback(p):
                    asyncio.run_coroutine_threadsafe(
                        self._update_progress(10 + int(p*0.6), f"⏳ Обработка: {p}%"),
                        self._loop
                    )

                return await self.processor.process_notebooks(
                    progress_callback=progress_callback
                )

            # Запускаем задачу с таймаутом
            process_task = asyncio.create_task(process_notebooks_task())
            while not process_task.done():
                await asyncio.sleep(0.1)
                yield [gr.Slider(value=self._progress), self._status]

            process_result = await process_task

            if not process_result.success:
                raise Exception(process_result.message)

            # Создаем задачу для создания базы
            async def create_db_task():
                def db_progress(p):
                    asyncio.run_coroutine_threadsafe(
                        self._update_progress(70 + int(p*0.3), f"⏳ Сохранение: {p}%"),
                        self._loop
                    )

                result = self.db_manager.create_database(
                    dataframe=process_result.details['dataframe'],
                    progress_callback=db_progress
                )
                return result

            db_task = asyncio.create_task(create_db_task())
            while not db_task.done():
                await asyncio.sleep(0.1)
                yield [gr.Slider(value=self._progress), self._status]

            result = await db_task

            if not result.success:
                raise Exception(result.message)

            await self._update_progress(100, "✅ База создана!")
            yield [gr.Slider(value=100), "✅ База создана!"]

        except Exception as e:
            logger.error(f"Ошибка создания базы: {str(e)}", exc_info=True)
            await self._update_progress(0, f"❌ Ошибка: {str(e)}")
            yield [gr.Slider(value=0, visible=False), f"❌ Ошибка: {str(e)}"]

    def render_ui(self) -> gr.Blocks:
        '''Создает комплексный интерфейс с вкладками'''
        with gr.Blocks(title="Ассистент Colab") as ui:
            # вкладка авторизации
            with gr.Tab("Авторизация"):
                self._render_auth_tab()

            # вкладка управления базой
            with gr.Tab("Управление базой"):
                self._render_database_tab()

            # вкладка запросов пользователя
            with gr.Tab("Запросы"):
                self._render_query_tab()

            # вкладка информации
            with gr.Tab("Об ассистенте"):
                self._render_info_tab()

            # вкладка логов
            with gr.Tab("Логи"):
                self._render_log_tab()

            return ui

    def _render_auth_tab(self):
        '''проверка статуса драйва'''
        with gr.Blocks():
            gr.Markdown("## Google Drive Status")
            status_html = gr.HTML()

        def check_drive_auth():
            """Проверяет подключение Google Drive"""
            if os.path.exists('/content/drive/MyDrive'):
                return gr.Markdown("✅ Google Drive подключен")
            return gr.Markdown("❌ Google Drive не подключен. Выполните `drive.mount('/content/drive')`")

        # В render_ui добавьте:
        with gr.Tab("Проверка"):
            check_drive_auth()

    def _render_database_tab(self):
        """Рендеринг вкладки базы данных"""
        with gr.Column():
            gr.Markdown("### Управление базой данных")

            progress_bar = gr.Slider(
                minimum=0, maximum=100, step=1,
                label="Прогресс", interactive=False, visible=False
            )

            status_output = gr.Textbox(
                label="Статус", interactive=False, lines=3
            )

            create_btn = gr.Button("Создать базу", variant="primary")

            # Добавляем callback для обновления UI
            def update_ui(progress, message):
                progress_bar.value = progress
                status_output.value = message

            self.add_progress_callback(update_ui)

            create_btn.click(
                fn=self._create_db,
                outputs=[progress_bar, status_output]
            )

    def _render_query_tab(self):
        '''Вкладка взаимодействия с ассистентом'''
        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("### Настройки ассистента")
                system_prompt = gr.Textbox(
                    label="Системный промпт",
                    value=self.assistant.current_system_prompt,
                    lines=5,
                    interactive=True
                )
                update_btn = gr.Button("Обновить промпт")

                gr.Markdown("### Ваш запрос")
                query_input = gr.Textbox(
                    label="Введите запрос",
                    placeholder="Какой ноутбук содержит...",
                    lines=3
                )
                submit_btn = gr.Button("Отправить", variant="primary")

            with gr.Column(scale=3):
                gr.Markdown("### Ответ ассистента")
                output = gr.Textbox(
                    label="Ответ",
                    interactive=False,
                    lines=10,
                    show_copy_button=True
                )
                token_count = gr.Number(
                    label="Использовано токенов",
                    interactive=False
                )

        # Обработчики событий
        def update_prompt(new_prompt):
            self.assistant.update_system_prompt(new_prompt)
            return "Промпт обновлен!"

        update_btn.click(
            update_prompt,
            inputs=system_prompt,
            outputs=output
        )

        def process_query(query):
            response, tokens = self.assistant.execute_query(query)
            return response, tokens

        submit_btn.click(
            process_query,
            inputs=query_input,
            outputs=[output, token_count]
        )
        query_input.submit(
            process_query,
            inputs=query_input,
            outputs=[output, token_count]
        )

    def _render_info_tab(self):
        '''Вкладка информации об Ассистенте'''
        gr.Markdown("### Ассистент для анализа ноутбуков Colab")
        gr.Markdown("""
        **Функционал:**
        - Автоматический анализ домашних работ
        - Поиск по сохраненным работам
        - интеграция с ChromaDB и LLM
        """)

    def _render_log_tab(self):
        with gr.Column():
            gr.Markdown("### Журнал событий")
            log_viewer = gr.Textbox(
                label="Логи",
                interactive=False,
                lines=20,
                value=self._read_logs()
            )
            refresh_btn = gr.Button("Обновить логи")

            def update_logs():
                return self._read_logs()

            refresh_btn.click(update_logs, outputs=log_viewer)

    def _read_logs(self, n_lines=100):
        try:
            log_file = Path("/content/logs/assistant.log")
            if not log_file.exists():
                return "Лог-файл не найден. Последние ошибки:\n" + self._get_recent_errors()

            with open(log_file, "r", encoding='utf-8') as f:
                lines = deque(f, maxlen=n_lines)
            return "".join(lines)
        except Exception as e:
            return f"Ошибка чтения логов: {str(e)}"

    def _get_recent_errors(self):
        """Собирает последние ошибки из памяти логгера"""
        if not hasattr(logger, 'handlers'):
            return "Нет информации об ошибках"

        buffer = []
        for handler in logger.handlers:
            if isinstance(handler, logging.StreamHandler):
                if hasattr(handler, 'stream'):
                    try:
                        handler.stream.seek(0)
                        buffer.append(handler.stream.read())
                    except (AttributeError, IOError):
                        continue
        return "\n".join(buffer) or "Нет записей об ошибках"

## Запуск интерфейса

In [ ]:
!rm -rf /content/drive/MyDrive/chroma_db

Инициализация компонентов

In [ ]:
embeddings = OpenAIEmbeddings(
    openai_api_key=API_KEY,
    openai_api_base=API_BASE,
    model="text-embedding-3-small"
)
llm_analyzer = LLMAnalyzer(openai_api_key=API_KEY, openai_api_base=API_BASE)
db_manager = ChromaDBManager(
    persist_dir="/content/drive/MyDrive/assistant/chroma_db",
    embeddings=embeddings
)
processor = NotebookProcessor(llm_analyzer)
assistant = Assistant(db_manager, llm_analyzer)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Здесь вы пройдёте авторизацию
from google.colab import auth
auth.authenticate_user()  # Токен для Google API

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import logging
from pathlib import Path

# Создаем папку для логов
log_dir = Path("/content/logs")
log_dir.mkdir(exist_ok=True)

# Настройка логгера
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_dir / "assistant.log"),  # Запись в файл
        logging.StreamHandler()  # Вывод в консоль
    ]
)
logger = logging.getLogger(__name__)

Создание интерфейса

In [ ]:
interface = GradioInterface(processor, db_manager, assistant)
interface.render_ui().launch(
    share=True,
    prevent_thread_lock=True,
    show_error=True,
    debug=True
    )

!tail -f /content/logs/assistant.log  # Вывод последних записей

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ab956f7ee8293887af.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://ab956f7ee8293887af.gradio.live


In [ ]:
!tail -f /content/logs/assistant.log  # Вывод последних записей

ERROR:__main__:Ошибка обработки ноутбука Терновцев "4.3. Домашняя работа": unindent does not match any outer indentation level (<unknown>, line 75)
ERROR:__main__:Ошибка обработки ноутбука 8.3 Доп. задания Задача 3: invalid syntax (<unknown>, line 1)
ERROR:__main__:Ошибка обработки ноутбука 8.3 Доп. задания. Задача 2: invalid syntax (<unknown>, line 1)
ERROR:__main__:Ошибка обработки ноутбука 8.3 доп. задания: invalid syntax (<unknown>, line 1)
ERROR:__main__:Ошибка обработки ноутбука 17.1. Проблема переобучения нейронных сетей: invalid syntax (<unknown>, line 215)
ERROR:__main__:Ошибка обработки ноутбука Копия блокнота "17.1. Проблема переобучения нейронных сетей": invalid syntax (<unknown>, line 215)
ERROR:__main__:Ошибка обработки ноутбука Копия блокнота "17.4. Домашняя работа": unexpected indent (<unknown>, line 1005)
ERROR:__main__:Ошибка обработки ноутбука 18.2. Обучение сверточной нейронной сети на ограниченном наборе данных.ipynb: unmatched ')' (<unknown>, line 208)
ERROR:__mai

^C


# Тесты отдельных кусков

Оставлю не все тесты, чтобы не запутаться

In [ ]:
from google.colab import userdata
import json

# Загружаем API-ключ из Secrets
try:
    F5AI_API_KEY = userdata.get('F5AI_API_KEY')
    F5AI_API_URL = "https://api.f5ai.ru/v1/models"

    # Выполняем запрос через requests (более надежно, чем curl)
    response = requests.get(
        F5AI_API_URL,
        headers={"X-Auth-Token": f"{F5AI_API_KEY}"}
    )

    if response.status_code == 200:
        data = response.json()

        # Красивое форматирование списка моделей
        print("═" * 50)
        print(f"{'ДОСТУПНЫЕ МОДЕЛИ F5AI':^50}")
        print("═" * 50)

        for model in data["response"]["models"]:
            status = "✅ Доступна" if model["available"] else "❌ Недоступна"
            print(f"\n▪ Модель: {model['code']}")
            print(f"  Статус: {status}")
            print(f"  Контекст: {model['context_window']} токенов")
            print(f"  Макс. вывод: {model['max_output']} токенов")
            if model["vision"]:
                print("  🔍 Поддержка vision")

        print("\n" + "═" * 50)
    else:
        print(f"Ошибка {response.status_code}: {response.text}")

except Exception as e:
    print(f"Произошла ошибка: {str(e)}")

══════════════════════════════════════════════════
              ДОСТУПНЫЕ МОДЕЛИ F5AI               
══════════════════════════════════════════════════

▪ Модель: gpt-3.5-turbo
  Статус: ❌ Недоступна
  Контекст: 16385 токенов
  Макс. вывод: 4096 токенов

▪ Модель: gpt-4-turbo
  Статус: ❌ Недоступна
  Контекст: 128000 токенов
  Макс. вывод: 4096 токенов

▪ Модель: gpt-4.1
  Статус: ✅ Доступна
  Контекст: 1047576 токенов
  Макс. вывод: 32768 токенов
  🔍 Поддержка vision

▪ Модель: gpt-4.1-mini
  Статус: ✅ Доступна
  Контекст: 1047576 токенов
  Макс. вывод: 32768 токенов
  🔍 Поддержка vision

▪ Модель: gpt-4.1-nano
  Статус: ✅ Доступна
  Контекст: 1047576 токенов
  Макс. вывод: 32768 токенов
  🔍 Поддержка vision

▪ Модель: gpt-4.5-preview
  Статус: ✅ Доступна
  Контекст: 128000 токенов
  Макс. вывод: 16384 токенов
  🔍 Поддержка vision

▪ Модель: gpt-4o
  Статус: ✅ Доступна
  Контекст: 128000 токенов
  Макс. вывод: 4096 токенов
  🔍 Поддержка vision

▪ Модель: gpt-4o-mini
  Статус: ✅ Досту

На F5AI поддержку моделей для эмбединга не нашел.

На DeepSeek Platform пополнить можно через вирутальную карту, но стоимость выпуска от 3000 рублей.

OpenAI без танцев с бубном не прикрутить в колаб с российским IP, даже если пополнить баланс через VPN.

Попался другой отлично задокументированный сервис AITunneL, плюсы - поддержка эмеббингов, понятная документация, детальные прайсы, доступное пополнение из России, полная совместимость с OpenAI API, поддержка langchain.
Буду использовать его, как альтернативу https://aitunnel.ru/

AST пробы для поиска подходящих выводов

In [ ]:
import ast

# Пример кода в виде строки
code = """
def example_function(x):
    if x > 0:
        return x
    else:
        return -x
"""

# Парсим код
tree = ast.parse(code)

# Функция для поиска return выражений
class ReturnVisitor(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print(f"Function '{node.name}' has the following return statements:")
        for n in node.body:
            if isinstance(n, ast.Return):
                print(f"  Return value: {ast.dump(n.value)}")
        self.generic_visit(node)

# Создаем экземпляр и запускаем обход
visitor = ReturnVisitor()
visitor.visit(tree)


Function 'example_function' has the following return statements:


In [ ]:
import ast

# Пример сложной функции
code = """
def complex_function(x, y):
    if x > 0:
        return x + y
    else:
        return y - (x * 2)

def another_function(a):
    return [i for i in range(a) if i % 2 == 0]
"""

# Парсим код
tree = ast.parse(code)

# Класс для нахождения функций
class FunctionVisitor(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print(f"Found function: '{node.name}'")
        print(f"  Arguments: {[arg.arg for arg in node.args.args]}")
        print(f"  Body:")
        for n in node.body:
            print(f"    {ast.dump(n)}")
        self.generic_visit(node)  # Продолжить обход

# Создаем экземпляр и запускаем обход
visitor = FunctionVisitor()
visitor.visit(tree)


Found function: 'complex_function'
  Arguments: ['x', 'y']
  Body:
    If(test=Compare(left=Name(id='x', ctx=Load()), ops=[Gt()], comparators=[Constant(value=0)]), body=[Return(value=BinOp(left=Name(id='x', ctx=Load()), op=Add(), right=Name(id='y', ctx=Load())))], orelse=[Return(value=BinOp(left=Name(id='y', ctx=Load()), op=Sub(), right=BinOp(left=Name(id='x', ctx=Load()), op=Mult(), right=Constant(value=2))))])
Found function: 'another_function'
  Arguments: ['a']
  Body:
    Return(value=ListComp(elt=Name(id='i', ctx=Load()), generators=[comprehension(target=Name(id='i', ctx=Store()), iter=Call(func=Name(id='range', ctx=Load()), args=[Name(id='a', ctx=Load())], keywords=[]), ifs=[Compare(left=BinOp(left=Name(id='i', ctx=Load()), op=Mod(), right=Constant(value=2)), ops=[Eq()], comparators=[Constant(value=0)])], is_async=0)]))


In [ ]:
import ast

# Пример сложной функции с использованием других функций и классов
code = """
class MyClass:
    def method_a(self, x):
        return x + 1

    def method_b(self, y):
        return self.method_a(y) * 2

def complex_function(x, y):
    instance = MyClass()
    result_a = instance.method_b(x)
    if result_a > 10:
        return result_a + y
    else:
        return y - (x * 2)
"""

# Парсим код
tree = ast.parse(code)

# Класс для нахождения функций и их содержимого
class FunctionVisitor(ast.NodeVisitor):
    def visit_FunctionDef(self, node):
        print(f"Found function: '{node.name}'")
        print(f"  Arguments: {[arg.arg for arg in node.args.args]}")
        print(f"  Body:")
        for n in node.body:
            self.visit(n)  # Обход каждого узла в теле функции
        self.generic_visit(node)  # Продолжить обход

    def visit_Call(self, node):
        # Обработка вызовов функций
        func_name = self.get_func_name(node)
        print(f"    Calls function: '{func_name}' with args {[ast.dump(arg) for arg in node.args]}")

    def visit_ClassDef(self, node):
        # Обработка классов
        print(f"Found class: '{node.name}'")
        self.generic_visit(node)  # Продолжить обход

    def get_func_name(self, node):
        # Получение имени функции из узла вызова
        if isinstance(node.func, ast.Name):
            return node.func.id
        elif isinstance(node.func, ast.Attribute):
            return f"{node.func.value.id}.{node.func.attr}"
        return "Unknown"

# Создаем экземпляр и запускаем обход
visitor = FunctionVisitor()
visitor.visit(tree)


Found class: 'MyClass'
Found function: 'method_a'
  Arguments: ['self', 'x']
  Body:
Found function: 'method_b'
  Arguments: ['self', 'y']
  Body:
    Calls function: 'self.method_a' with args ["Name(id='y', ctx=Load())"]
    Calls function: 'self.method_a' with args ["Name(id='y', ctx=Load())"]
Found function: 'complex_function'
  Arguments: ['x', 'y']
  Body:
    Calls function: 'MyClass' with args []
    Calls function: 'instance.method_b' with args ["Name(id='x', ctx=Load())"]
    Calls function: 'MyClass' with args []
    Calls function: 'instance.method_b' with args ["Name(id='x', ctx=Load())"]


In [ ]:
import ast

# Пример кода, в котором нужно найти функции и классы
code = """
class MyClass:
    def method_a(self, x):
        return x + 1

    def method_b(self, y):
        return self.method_a(y) * 2

def complex_function(x, y):
    instance = MyClass()
    result_a = instance.method_b(x)
    if result_a > 10:
        return result_a + y
    else:
        return y - (x * 2)
"""

# Парсим код
tree = ast.parse(code)

# Класс для нахождения функций и классов
class FunctionAndClassFinder(ast.NodeVisitor):
    def __init__(self):
        self.functions = []
        self.classes = []

    def visit_FunctionDef(self, node):
        self.functions.append(node.name)
        self.generic_visit(node)  # Продолжить обход

    def visit_ClassDef(self, node):
        self.classes.append(node.name)
        self.generic_visit(node)  # Продолжить обход

# Создаем экземпляр и запускаем обход
finder = FunctionAndClassFinder()
finder.visit(tree)

# Выводим найденные функции и классы
print("Найденные функции:")
for func in finder.functions:
    print(func)

print("\nНайденные классы:")
for cls in finder.classes:
    print(cls)


Найденные функции:
method_a
method_b
complex_function

Найденные классы:
MyClass


In [ ]:
import ast

# Пример кода, в котором нужно найти функции и классы
code = """
class MyClass:
    def method_a(self, x):
        return x + 1

    def method_b(self, y):
        return self.method_a(y) * 2

def complex_function(x, y):
    instance = MyClass()
    result_a = instance.method_b(x)
    if result_a > 10:
        return result_a + y
    else:
        return y - (x * 2)
"""

# Парсим код
tree = ast.parse(code)

# Класс для нахождения функций и классов
class FunctionAndClassFinder(ast.NodeVisitor):
    def __init__(self):
        self.functions = {}
        self.classes = {}

    def visit_FunctionDef(self, node):
        # Получаем тело функции
        function_body = ast.get_source_segment(code, node)
        self.functions[node.name] = function_body
        self.generic_visit(node)  # Продолжить обход

    def visit_ClassDef(self, node):
        # Получаем тело класса
        class_body = ast.get_source_segment(code, node)
        self.classes[node.name] = class_body
        self.generic_visit(node)  # Продолжить обход

# Создаем экземпляр и запускаем обход
finder = FunctionAndClassFinder()
finder.visit(tree)

# Выводим найденные функции и классы с телами
print("Найденные функции:")
for func_name, func_body in finder.functions.items():
    print(f"{func_name}:\n{func_body}\n")

print("Найденные классы:")
for class_name, class_body in finder.classes.items():
    print(f"{class_name}:\n{class_body}\n")


Найденные функции:
method_a:
def method_a(self, x):
        return x + 1

method_b:
def method_b(self, y):
        return self.method_a(y) * 2

complex_function:
def complex_function(x, y):
    instance = MyClass()
    result_a = instance.method_b(x)
    if result_a > 10:
        return result_a + y
    else:
        return y - (x * 2)

Найденные классы:
MyClass:
class MyClass:
    def method_a(self, x):
        return x + 1

    def method_b(self, y):
        return self.method_a(y) * 2



Последний вариант - то что искал

тест создания базы

In [ ]:
import pandas as pd

test_data = pd.DataFrame([
    {
        "название": "Тестовый ноутбук 1",
        "заголовки": "# Введение\n## Задача 1",
        "описание": "Функция: calculate_sum - складывает числа",
        "итог": "Итог: успешное тестирование",
        "библиотеки": "numpy, pandas",
        "классы и функции": "def calculate_sum(a, b)"
    },
    {
        "название": "Тестовый ноутбук 2",
        "заголовки": "# Анализ данных\n## Визуализация",
        "описание": "Функция: plot_graph - строит график",
        "итог": "Итог: графики построены",
        "библиотеки": "matplotlib",
        "классы и функции": "def plot_graph(data)"
    }
])
def test_db_creation():
    try:
        # Путь для тестовой базы (временная папка)
        test_db_path = "/content/test_chroma_db"

        # Инициализация
        embeddings = OpenAIEmbeddings(API_KEY)
        db_manager = ChromaDBManager(
            persist_dir=test_db_path,
            embeddings=embeddings
        )

        # Создание тестовой базы
        result = db_manager.create_database(test_data)

        # Проверка результатов
        if result.success:
            print("✅ Тест пройден. База создана успешно!")
            print(f"Путь: {test_db_path}")
            print(f"Сообщение: {result.message}")

            # Дополнительная проверка - поиск в базе
            search_result = db_manager.search_documents("график")
            print("\nРезультаты поиска по запросу 'график':")
            for doc in search_result.details["results"]:
                print(f"- {doc['document'][:100]}...")
        else:
            print("❌ Ошибка создания базы:")
            print(result.message)
            if result.details:
                print("Детали:", result.details)

    except Exception as e:
        print("⚠️ Критическая ошибка:")
        print(f"Тип: {type(e).__name__}")
        print(f"Сообщение: {str(e)}")

# Запуск теста
test_db_creation()

⚠️ Критическая ошибка:
Тип: OpenAIError
Сообщение: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


# Ошибки

Долго бился с апи ключами, langchain не хотел принимать их - изменил импорты на

`from langchain_openai import ChatOpenAI, OpenAIEmbeddings`

 ошибка пропала.

При инициализации столкнулся с проблемой, что хрома требует директорию для себя еще до подключения гугл драйв, что меня не устраивает. Выход - использовать ленивую инициализацию

Не отрисовывались элементы в интервейсе на вкладке Авторизация, причина - в Gradio при использовании контекстных менеджеров (with) элементы должны быть либо привязаны к переменной, либо метод должен возвращать сам блок. добавил блок в `return`

При попытке авторизации на драйве через интерфейс градио появляется ошибка без деталей (?). при авторизации с помощью магического метода колаб всплывает окно с запросом доступа, чего не было через интерфейс градио, и только после предоставления полного доступа к любым сервисам драйва диск наконец подключился. При использовании в личных целях это не несет угрозы, но если будет публичный доступ - авторизацию необходимо будет переделывать на OAuth и API Google. ВРеменное решение - заменил на вкладке авторизацию на проверку, а авторизацию провожу до запуска интерфейса.

Здесь не перечислены остальные ошибки, встреченные в процессе написания, т.к. на это бы ушло столько же времени, как и на проект.

# Итог

На данной странице получилось реализовать только часть проекта, и то с критическими ошибками в исполнении кода. здесь код уже настолько пересобирался и засорился, что я решил переписать код заново в чистом виде. ссылка на новый код https://colab.research.google.com/drive/1AOTAtniT0CWkbGgqSLn-rUzAxHD1drrX?usp=sharing